In [16]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.datasets import fetch_california_housing
import pandas as pd 
import numpy as np
from urllib.parse import urlparse

In [17]:
housing = fetch_california_housing()

housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

# Prepare the dataset

In [18]:
data = pd.DataFrame(housing.data,columns=housing.feature_names)
# add target value into this data
data['Price'] = housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# Train test split

In [19]:
# Features
X = data.drop(columns=['Price'])

#Target value
Y =data['Price']


print(len(X))
print(len(Y))

20640
20640


In [20]:
# Split data into train and test sets
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

print(len(X_train))
print(len(X_test))



16512
4128


In [21]:
# Hyperparameter tuning using Grid Searchcv

def hyperparameter_tune(X_train,y_train,grid):
    rf=RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf,param_grid=grid,cv=4,n_jobs=-1,
                               verbose=3,
                               scoring='neg_mean_squared_error'
                               )
    grid_search.fit(X_train,y_train)
    
    return grid_search

# https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
from mlflow.models import infer_signature
#infer_signature  : used to enable both the MLflow Tracking UI and Model Registry UI to clearly display the model's required inputs, outputs, and parameters.

signature = infer_signature(X_train,y_train) 

# Hyperparameters

param_grid={
    'n_estimators':[100,200],
    'max_depth' : [10,20,None],
    'min_samples_split':[2,5],
    'min_samples_leaf':[1,2]
}

In [ ]:
with mlflow.start_run():
    grid_search = hyperparameter_tune(X_train,y_train,param_grid)
    
    ## get best model 
    best_model = grid_search.best_estimator_
    
    ##evaluate the best model
    y_prid = best_model.predict(X_test)
    mse=mean_squared_error(y_test,y_prid)
    
    ## log Best model and metrics
    
    mlflow.log_params("best_n_estimators",grid_search.best_params_['n_estimators'])
    mlflow.log_params("best_max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_params("best_min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_params("best_min_samples_leaf",grid_search.best_params_['min_samples_leaf'])
    mlflow.log_params("mse",mse)
    
    #Tracking Uri
    
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    
    tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
    
    if tracking_uri_type_store !='file':
        mlflow.sklearn.log_model(best_model,"model",registered_model_name="Best Randomforest Model")
        
    else:
        mlflow.sklearn.log_model(best_model,"model",signature=signature)
        
    print(f"Best Hyperparameters : {grid_search.best_params_}")
    print(f"Mean Squred Error  : {mse}")

Fitting 4 folds for each of 24 candidates, totalling 96 fits


ValueError: Invalid parameter 'min_sample_leaf' for estimator RandomForestRegressor(max_depth=10). Valid parameters are: ['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'monotonic_cst', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].